 ###  Memory Allocators

 OpenMP memory allocators can be used to allocate memory with specific allocator traits.  In the following example an OpenMP allocator is used to specify an alignment for arrays  _x_  and  _y_ . The general approach for attributing traits to variables allocated by OpenMP is to create or specify a pre-defined  _memory space_ , create an array of  _traits_ , and then form an  _allocator_  from the memory space and trait. The allocator is then specified in an OpenMP allocation (using an API  _omp_alloc()_  function for C/C++  and an  `allocate`  directive for Fortran  in the allocators.1 example).

 In the example below the  _xy_memspace_  variable is declared and assigned the default memory space ( _omp_default_mem_space_ ). Next, an array for  _traits_  is created. Since only one trait will be used, the array size is  _1_ . A trait is a structure in C/C++ and a derived type in Fortran, containing 2 components: a key and a corresponding value (key-value pair). The trait key used here is  _omp_atk_alignment_  (an enum for C/C++ and a parameter for Fortran) and the trait value of 64 is specified in the  _xy_traits_  declaration. These declarations are followed by a call to the  _omp_init_allocator()_  function to combine the memory space ( _xy_memspace_ ) and the traits ( _xy_traits_ ) to form an allocator ( _xy_alloc_ ).

 
In the C/C++  the API   _omp_allocate()_  function is used  
to allocate space, similar to  _malloc_ , except that the allocator  
is specified as the second argument.   
In Fortran an API allocation function is not available.  
An  `allocate`  construct is used (with  _x_  and  _y_   
listed as the variables to be allocated), along 
with an  `allocator`  clause (specifying the  _xy_alloc_  as the allocator) 
for the following Fortran  _allocate_  statement.

 In the C/C++  the API   _omp_allocate()_  function is used to allocate space, similar to  _malloc_ , except that the allocator is specified as the second argument. In Fortran an  `allocate`  directive is used to specify an allocator for a following Fortran  _allocate_  statement. A variable list may be supplied if the allocator is to be applied to a subset of variables in the Fortran allocate statement. Specifying the complete list is optional. Here, the  _xy_alloc_  allocator is specified in the  `allocator`  clause, and the set of all variables used in the allocate statement is specified in the list.

 
'for a following Fortran allocation statement' (no using 'immediately' here) 

In [ ]:

/*
* name: allocators.1c
* type: C
* version: omp_5.0
*/
#include    <omp.h>
#include  <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#define N 1000

int main()
{
   float  *x, *y;
   float s=2.0;

omp_memspace_handle_t  xy_memspace = omp_default_mem_space;
omp_alloctrait_t       xy_traits[1]={omp_atk_alignment, 64};
omp_allocator_handle_t xy_alloc    = omp_init_allocator(xy_memspace,1,xy_traits);


   x=(float *)omp_alloc(N*sizeof(float), xy_alloc);
   y=(float *)omp_alloc(N*sizeof(float), xy_alloc);

   if( ((intptr_t)(y))%64 != 0 || ((intptr_t)(x))%64 != 0 )
   { printf("ERROR: x|y not 64-Byte aligned\n"); exit(1); }

   #pragma omp parallel
   {
      #pragma omp for simd simdlen(16) aligned(x,y:64)
      for(int i=0; i<N; i++){ x[i]=i+1; y[i]=i+1; } // initialize

      #pragma omp for simd simdlen(16) aligned(x,y:64)
      for(int i=0; i<N; i++) y[i] = s*x[i] + y[i];
    }


   printf("y[0],y[N-1]: %5.0f %5.0f\n",y[0],y[N-1]); //output: y...   3  3000

   omp_free(x, xy_alloc);
   omp_free(y, xy_alloc);
   omp_destroy_allocator(xy_alloc);

   return 0;
}



In [ ]:

! name: allocators.1f90
! type: F-free
! version: omp_5.0

program main
 use omp_lib

 integer, parameter :: N=1000, align=64
 real, allocatable  :: x(:),y(:)
 real               :: s = 2.0e0
 integer            :: i

 integer(omp_memspace_handle_kind ) :: xy_memspace = omp_default_mem_space
 type(   omp_alloctrait           ) :: xy_traits(1) = &
                                          [omp_alloctrait(omp_atk_alignment,64)]
 integer(omp_allocator_handle_kind) :: xy_alloc

   xy_alloc   =    omp_init_allocator(   xy_memspace, 1, xy_traits)

   !$omp allocate(x,y) allocator(xy_alloc)
   allocate(x(N),y(N))
                         !! loc is non-standard, but found everywhere
                         !! remove these lines if not available
   if(modulo(loc(x),align) /= 0 .and. modulo(loc(y),align) /=0 ) then
      print*,"ERROR: x|y not 64-byte aligned"; stop
   endif

   !$omp parallel

      !$omp do simd simdlen(16) aligned(x,y: 64) !! 64B aligned
      do i=1,N  !! initialize
        x(i)=i
        y(i)=i
      end do

      !$omp do simd simdlen(16) aligned(x,y: 64) !! 64B aligned
      do i = 1,N
         y(i) = s*x(i) + y(i)
      end do

   !$omp end parallel

   write(*,'("y(1),y(N):",2f6.0)') y(1),y(N) !!output: y... 3. 3000.

   deallocate(x,y)
   call omp_destroy_allocator(xy_alloc)

end program


